# Graphing Code to Load Data and Plot It Live

## Imports

In [1]:
import plotly as py
import plotly.tools as tls
from plotly import graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
import pandas as pd
import datetime
import Processing_Functions_Tracking
import json
import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html

from ast import literal_eval

## Initialising Dash and Plotly

In [2]:
app = dash.Dash(__name__)

#Plotly offline mode
init_notebook_mode(connected=True)

timer = 0

In [3]:
#Function to check if any of the items contain an array
def array_check(item,timer):
    item = item[0]
    print(item)
    if(isinstance(item, str)):
        print(item)
        item = literal_eval(item)
        return round(item[timer],2)
    elif(isinstance(item, list)):

        return round(item[-1],2)
    else:

        return round(float(item),2)

# Loading Data

In [4]:

path = path = "../Experiments/Detecting Dust Devils - Testing Tracking Functionality with R Added Dust Probability/60 Robots/"
store_robots = np.load(path + "Robots.npy")

with open(path + "dust.txt", "r") as f:
     store_dust = json.load(f)


constants = pd.read_excel(path + "Constants.xlsx", index_col=0)
min_neighbours = np.load(path + "Minimum Distance to Neighbours.npy")
cluster_average = np.load(path + "Cluster Average.npy")
total_collision = np.load(path + "Measurement Events Count.npy")
total_detection = np.load(path + "Number of Dust Devils Detected.npy")
total_dust = np.load(path + "Number of Dust Devils Generated.npy")

In [5]:
temp = store_robots[0]
print(temp)
print(temp)

[[ 350.7961961   352.79435725  351.15761046 ...  348.46427961
   348.92998265  346.95847738]
 [  54.81259012   56.69472243   54.81197165 ...   51.18412502
    50.28392443   49.55260569]
 [ 499.63110245  499.63110245  499.63110245 ...  499.65380709
   499.65380709  499.65380709]
 ...
 [ 237.61076957  238.26245342  238.20094736 ...  231.16163128
   233.08362882  231.65790546]
 [-190.6261893  -189.79376154 -191.28298743 ... -201.77166982
  -201.19155666 -202.30343714]
 [ 221.53596     223.31806667  221.51411845 ...  211.5933808
   210.25183997  211.77112014]]
[[ 350.7961961   352.79435725  351.15761046 ...  348.46427961
   348.92998265  346.95847738]
 [  54.81259012   56.69472243   54.81197165 ...   51.18412502
    50.28392443   49.55260569]
 [ 499.63110245  499.63110245  499.63110245 ...  499.65380709
   499.65380709  499.65380709]
 ...
 [ 237.61076957  238.26245342  238.20094736 ...  231.16163128
   233.08362882  231.65790546]
 [-190.6261893  -189.79376154 -191.28298743 ... -201.7716698

In [6]:
#the update layout function
def update_layout():
    #contains the graph, the interval timing and the live updating table
    app.layout = html.Div(children=[html.Div(
    [
    dcc.Graph(id = 'live_graph'),
    dcc.Interval(
        id = 'graph-update',
        interval = 100, #milliseconds
        n_intervals = 0
        ),
    ],style = {'display': 'inline-block'} ),
        html.Div(
            [
                dcc.Graph(id = 'live_table'),
                dcc.Interval(
                id = 'variable-update',
                interval = 100, #milliseconds
                n_intervals = 0
                )    
                 ],
        style = {'display': 'inline-block','vertical-align': 'top' }    
    )],style={'width': '100%', 'display': 'inline-block'})

update_layout()

In [7]:
#callback is the updating process
@app.callback(Output('live_graph','figure'),
              [Input('graph-update', 'n_intervals')])

#the actual update for the graph
def update_graph(n):
    #initialising global variables
    global timer

    position_robot = store_robots[:,:,:]
    position_dust = store_dust[timer]
    #creating scatter plot of robots
    data = go.Scatter(
        x=list(position_robot[0]),
        y=list(position_robot[1]),
        name = 'Robots',
        mode = 'markers',
    )
    
    #creating the plotly figure with the robot data
    fig = go.Figure(
        { "data": data, "layout": go.Layout(yaxis=dict(range=[-1000, 1000]),xaxis = dict(range=[-1000,1000]))
        }
    )

    fig.add_trace(go.Scatter(
          x= position_dust[0],
          y= position_dust[1],
          name='Dust Devil',
          mode = 'markers',
          marker_color='rgba(255, 182, 193, .9)',
      ))
    
    #updating layout with circles and different formatting'''
    fig.update_layout(title="<b>Physics Based Swarm Experiment </b>",
    title_x=0.5,
    xaxis_title="X Position (m)",
    yaxis_title="Y Position (m)",
    margin=dict(
        t=50, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
         # bottom margin: 10px
        l=10, # left margin: 10px
        r=10, # right margin: 10px
    ),
    height=900,width=1150,
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 50
    ),
                      yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 50
    )
                     )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    #incrementing timer
    timer = timer + 1
    
    #updating the layout
    update_layout()
    return fig

#app callback for the table
@app.callback(Output('live_table','figure'),
              [Input('variable-update', 'n_intervals')])

def update_variables(n):
    fig = Processing_Functions_Tracking.table_figure(store_robots,timer,1,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
               # http://localhost:8888/notebooks/Documents/University%20of%20Bristol/Masters/Dissertation/Git_Code/Simulation/Types/Grapher%20Review.ipynb#
    fig.update_layout(width = 650, height = 800,margin=dict(
    t=200, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
        b=10, # bottom margin: 10px
        l=10, # left margin: 10px
        r=10,
    ))
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    return fig

In [ ]:
#runs by default
if __name__ == '__main__':
    #running the app server for the simulator
    app.run_server(debug=False, host = '127.0.0.1')

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Sep/2021 02:18:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:18:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:18:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:19:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:19:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:19:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:19:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:19:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:19:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:19:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:19:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 

127.0.0.1 - - [07/Sep/2021 02:20:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:20:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:20:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:20:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:20:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:20:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:20:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:20:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:20:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:20:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:20:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2021 02:20:19] "POST /_dash-update-component HTTP/1.1" 200 -
